# 1 Getting started

Please make sure you have the proper environment and all necessary packages installed. Please refer to the README file for setting this up. 
Also, **remember to choose the conda env:ricse_bw25 as kernel**.

Please note that this notebook is designed to quickly help you complete LCA with BW25. However there is much to learn about data types 

This notebook has excerpts form the Brightway Book [here](https://learn.brightway.dev/en/latest/content/chapters/BW25/BW25_introduction.html#introductory-note) 

For shortcuts, check the BW cheatsheet [here](https://docs.brightway.dev/en/latest/content/cheatsheet/index.html)


# 2 Understanding BW25 structure
After you have BW25 up and running, you need to understand the BW2 data structure. Please read about it in the BW25 site [here](https://docs.brightway.dev/en/latest/content/overview/structure.html).

# 3 Importing needed packages
As you may know, before you are able to import the packages we will later use. For the moment:

In [62]:
import bw2analyzer as bwa
import bw2calc as bc
import bw2data as bd
import bw2io as bi

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# 4 Set up a project with biosphere and LCIA methods
We need to set a project with the Biosphere3 and LCIA methods databases in it. 

In here you have only the very basics.
More commands for projects: https://docs.brightway.dev/en/latest/content/cheatsheet/projects.html

You can see what projects you have already doing:


In [3]:
sorted(bd.projects) 

[Project: default, Project: SAPP_chair, Project: Supdup Exercise 7 project]

You can see the path to the place where the projects are saved doing:

In [2]:
bd.projects.dir

WindowsPath('C:/Users/user/AppData/Local/pylca/Brightway3/default.c21f969b5f03d33d43e04f8f136e7682')

Check the project that is currently active:

In [4]:
bd.projects.current

'default'

If you did not active any project, the cell above will return the default project.

There is a way of creating a project already with the Biosphere3 and the methods library in it. We will create the RICSE_prj project and will choose the tag of the Db we will later import: Ecoinvent 3.9.1.

In [2]:
bi.remote.install_project('ecoinvent-3.9.1-biosphere', 'RICSE_prj')

Restoring project backup archive - this could take a few minutes...
Restored project: RICSE_prj


'RICSE_prj'

There are  few alternatives to the project_tag, is one of:
* ecoinvent-3.10-biosphere
* ecoinvent-3.8-biosphere
* ecoinvent-3.9.1-biosphere
* forwast
* USEEIO-1.1

The ecoinvent biosphere projects also include LCIA impact categories.

Now we still need to set the **RICSE_prj** project as the active project:

In [3]:
bd.projects.set_current(name='RICSE_prj')

23:27:31 [info     ] Applying automatic update: 4.0 database search directories FTS5
23:27:31 [info     ] Reindexing database biosphere3


Just in case you need to delete a project

In [2]:
bd.projects.delete_project("RICSE_prj",delete_dir=True)

ValueError: RICSE_prj is not a project

You can now come back to the previous lines of code and check that your project has been added to your project list!!

# 5 Set databases
We have just created an empty project. If it is the first time working with it, it does not yet contain any inventory data. If you have worked with it, it will have DBs.

More information about databases here: https://docs.brightway.dev/en/latest/content/cheatsheet/databases.html

We can check the DBs of our project now:

In [4]:
bd.databases

Databases dictionary with 1 object(s):
	biosphere3

In case you would like to check the methods in your DB, you can do:

In [ ]:
sorted(bd.methods)

## 5.1 Importing Ecoivent

Now we import the Ecoinvent data. We will start with the UP data and then upload the LCI data
Remember to unzip the file. Then, provide the path to the unzipped folder **datasets** in the code below. 

You can find more information about data importing here:
https://docs.brightway.dev/en/latest/content/cheatsheet/importing.html#

We are importing first the Unit Process DB. Even thought we can create foreground processes ourselves, it will come in hando to have the option of using and modifying a process from the UP database.
**This process takes about 2 minutes in Cristina's computer**


In [5]:
filedir=r'C:\Users\user\UAB\LIVENlab - ENBIOSdev - ENBIOSdev\Ecoinvent_RAW\ecoinvent391_cutoff_DONOTEDIT\datasets'
importer = bi.SingleOutputEcospold2Importer(
    dirpath=filedir,
    db_name='ei_391_cutoff'
)
importer.apply_strategies()
importer.write_database()

Extracting XML data from 21238 datasets
Extracted 21238 datasets in 57.76 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: 

100%|██████████| 21238/21238 [00:51<00:00, 414.02it/s]


23:29:56 [info     ] Vacuuming database            
Created database: ei_391_cutoff


Brightway2 SQLiteBackend: ei_391_cutoff

In [ ]:
#to delete a DB
#del bd.databases['<database_name>']

## 5.2 Check the lenght of a database
In you are unsure about the proper importing of the ecoinvent DBs, check here.

In [6]:
db = bd.Database('ei_391_cutoff')
# Get the length of the database
length = len(db)
print(f"The database contains {length} datasets.")

The database contains 21238 datasets.


We can also draw the interactions of the Database. This is how the technosphere links of a database look like:

In [ ]:
ei = bd.Database('ei_391_cutoff')
ei.graph_technosphere()


## 5.3 Understanding nodes
Let's examine what's inside ecoinvent. This is how we search inside any database.

You can produce nodes and assign their related exchanges manually. For a cheatsheet: 
https://docs.brightway.dev/en/latest/content/cheatsheet/inventory.html

In [ ]:
bd.Database('ei_391_cutoff').search('electricity production, wind', limit=100)

And this is how we isolate a node of the database:

In [ ]:
mynode = bd.Database('ei_391_cutoff').get(name = 'electricity production, wind, >3MW turbine, onshore', location = 'DE')
mynode

In [ ]:
myrandomnode = bd.Database('ei_391_cutoff').random()
myrandomnode

Nodes have the form of complicated structures. You can see the attributes of a node by doing:

In [ ]:
myrandomnode.as_dict()

See how you have a first part with metadata about the scope, the codes and units and other information and then a set of exchanges with their types. We can also see the key f a node formed by the database name and the code of the node.

In [ ]:
myrandomnode.key

## 5.4 Understanding edges
Edges are the connections between nodes. They can be of different types including:
* Technosphere: the inputs to a process
* Biosphere: the emissions to the environment or input from the environment
* Production: the production of the reference product

Let's see the differences between the edges of a node form the UP and the LCI databases.

This is how you can see the inputs of a node:

In [ ]:
list(myrandomnode.technosphere()) 

This is how you see the products of a node:

In [ ]:
list(myrandomnode.production()) 

This is how you see the exchanges with nature:

In [ ]:
list(myrandomnode.biosphere()) 

This is how you see who is consuming the products of the node (Downstream):

In [ ]:
list(myrandomnode.consumers())

This is how you change an exchange's attribute:

In [ ]:
#myrandomnode.exchanges()[0]['amount'] = 2.5

And you can also save the changes:

In [ ]:
#myrandomnode.save()

# 6 Producing your own LCA system

We will use a simple example to understand how to create a node from scratch and a system with their nodes and edges in BW25. Let's assume that we would like to model the Spanish energy mix and calculate the impacts associated to it. Our scope will include the following inputs to the energy mix:
* 40% of wind energy
* 30% of photovoltaic energy
* 20% of the electricity comes from storage

We need to start creating and registering a new empty DB, where we will update our new processes, we will call it **myfg_db**. After we have created it, we can check that it is included in the list of BDs to be sure.

In [7]:
myfg_db = bd.Database('myfg_db')
myfg_db.register()


We also might want to check that's already in our myfg_db database list:

In [8]:
myfg_db = bd.Database('myfg_db')
for activity in myfg_db:
    print(f"Activity: {activity['name']} ({activity['location']})")

Or we might need to delete it:

In [ ]:
#del bd.databases['myfg_db']

## 6.1 Create Activities
Now we can create our reference node for the energy mix that we wil name **enermix**. As we saw above a node has many attributes that we need to add, including the name, the code, the categories, the type, the unit and the location of the activity. The code must be unique for each activity.


In [30]:
enermix = bd.Database('myfg_db').new_node(
    code='energy mix SP ricse', 
    name="Energy Mix Spain RICSE", 
    categories=('energy',),
    unit='kilowatt hour',
    location= 'ES')
enermix.save()

We can now check the contents of the node

In [28]:
for activity in myfg_db:
    print(f"Activity: {activity['name']} ({activity['location']})")
   

In [31]:
enermix=bd.Database('myfg_db').get('energy mix SP ricse')
enermix.as_dict()

{'database': 'myfg_db',
 'code': 'energy mix SP ricse',
 'name': 'Energy Mix Spain RICSE',
 'categories': ('energy',),
 'unit': 'kilowatt hour',
 'location': 'ES',
 'id': 25948}

**Project TIP**: You can create any other nodes you need to create manually for your model in the same way.

And if you need to delete your node:

In [27]:
#enermix=bd.Database('myfg_db').get('energy mix SP ricse')
#enermix.delete()

## 6.2 Create exchanges
We are now going to add the inputs of wind, photovoltaics and battery provided electricity in our mix using nodes from the ecoinvent database.

As there are many processes with the same name and different location of with different technologies, we want to make sure we add the exact process we need. We are using this search engine to define our three nodes and also start by adding from the UP database, for the moment.

You can use the facet parameter to filter the search results. For example, if you want to search for a process in a specific location, you can use the location facet.

In [ ]:
bd.Database('ei_391_cutoff').search('battery', limit=100, facet='location')

In [32]:
windup = bd.Database('ei_391_cutoff').get(name = 'electricity production, wind, 1-3MW turbine, onshore', location = 'ES')
PVup = bd.Database('ei_391_cutoff').get(name = 'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted', location = 'ES')
batteryup = bd.Database('ei_391_cutoff').get (name = 'battery production, lead acid, rechargeable, stationary', location = 'US')
batteryup

'battery production, lead acid, rechargeable, stationary' (kilogram, US, None)

Note that the batteries in ecoinvent are given in KG. This means that in order for us to include electricity frm battery we have to include the conversion factor from KG to KWh. A typical value for the energy density of a battery is 0.1 KWh/Kg. and in each cycle it produces.

Remember that we are modelling for 1 Kwh of electricity produced by the energymix activity.

In [33]:
prodedge = enermix.new_edge(
    amount=1, 
    type='production',
    input= enermix)
prodedge.save()
windedge = enermix.new_edge(
    amount=0.4, 
    code = 'windup',    
    type='technosphere',
    input= windup)
windedge.save()

PVedge = enermix.new_edge(
    amount=0.3, 
    code = 'PVup',    
    type='technosphere',
    input= PVup) 
PVedge.save()

batterydensity_kgkwh= 0.1
Batedge =enermix.new_edge(
    amount=0.2/batterydensity_kgkwh, 
    code = 'battup',    
    type='technosphere',
    input= batteryup)
Batedge.save()

We can now check the exchanges of the node:

In [34]:
list(enermix.technosphere()) 

[Exchange: 0.4 kilowatt hour 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, ES, None) to 'Energy Mix Spain RICSE' (kilowatt hour, ES, ('energy',))>,
 Exchange: 0.3 kilowatt hour 'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted' (kilowatt hour, ES, None) to 'Energy Mix Spain RICSE' (kilowatt hour, ES, ('energy',))>,
 Exchange: 2.0 kilogram 'battery production, lead acid, rechargeable, stationary' (kilogram, US, None) to 'Energy Mix Spain RICSE' (kilowatt hour, ES, ('energy',))>]

Of the flows going to or coming from the environment:

In [15]:
list(enermix.biosphere())

[]

If you need to delete exchanges: 

In [ ]:
#my_edge = enermix.exchanges(name = )
#my_edge.delete()

# 7 Inventory and  Impact Assessment

In BW25 there is a function to calculate LCIA results for a single functional unit and a single method. This is the `LCA` class. However the interesting result of a LCA is usually the contribution and comparison of results for different methods, indicators and functional units. This is done with the `MultiLCA` class.

## 7.1 Examining LCIA methods
Let's first examine again the methods you have in your project: 

In [25]:
sorted(bd.methods)

[('CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'),
 ('CML v4.8 2016',
  'ecotoxicity: freshwater',
  'freshwater aquatic ecotoxicity (FAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: marine',
  'marine aquatic ecotoxicity (MAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity (TETP inf)'),
 ('CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)'),
 ('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)'),
 ('CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ultimate reserves)'),
 ('CML v4.8 2016',
  'ozone depletion',
  'ozone layer depletion (ODP steady state)'),
 ('CML v4.8 2016',
  'photochemical oxidant formation',
  'photochemical oxidation (h

We now will have to make a selection of methods we want to include in our assessment. 

## 7.2 Single LCIA scores
This is already enough to do an single LCA. We will calculate the impact of 1 Kwh produced by our energy mix comparing both the UP and the LCI databases. We will use one of the Recipe Midpoint methods for this.

We have left 1 as the amount but we could also be changing this to any other value if we opt for a different functional unit.

In [35]:
mymethod=('ReCiPe 2016 v1.03, midpoint (E)',
  'climate change',
  'global warming potential (GWP1000)')

my_functional_unit = {enermix: 1}

lcaup = bc.LCA(demand=my_functional_unit, method=mymethod)
lcaup.lci()
lcaup.lcia()
upscore = lcaup.score
unit = bd.Method(mymethod).metadata["unit"]
print (upscore, unit)

4.596451056867587 kg CO2-Eq


We can also see the characterization matrix and the impact categories:

In [39]:
rowscolsup = lcaup.characterization_matrix.shape
charmatrixup = lcaup.characterization_matrix

We can now visualize the characterization matrix

In [41]:
charmatrix_df = pd.DataFrame(lcaup.characterization_matrix.toarray())

# Display the DataFrame as a table
charmatrix_df

,0,1,2,3,4,5,6,7,8,9,...,2410,2411,2412,2413,2414,2415,2416,2417,2418,2419
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Here is the matrix showing in rows the 

## 7.3 Multi LCIA scores
This is quite tedious and we want to automatize the calculation of the scores for all the methods we have in our project. 
We will start by identifying the methods we would like ot use for the assessment:

In [57]:
my_functional_unit = {enermix: 1}
my_methods = [method for method in bd.methods if method[0] == 'ReCiPe 2016 v1.03, midpoint (H)']
my_methods

[('ReCiPe 2016 v1.03, midpoint (H)',
  'acidification: terrestrial',
  'terrestrial acidification potential (TAP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'climate change',
  'global warming potential (GWP1000)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'ecotoxicity: freshwater',
  'freshwater ecotoxicity potential (FETP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'ecotoxicity: marine',
  'marine ecotoxicity potential (METP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity potential (TETP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'energy resources: non-renewable, fossil',
  'fossil fuel potential (FFP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'eutrophication: freshwater',
  'freshwater eutrophication potential (FEP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'eutrophication: marine',
  'marine eutrophication potential (MEP)'),
 ('ReCiPe 2016 v1.03, midpoint (H)',
  'human toxicity: carcinogenic',
  'human toxicity potential (HTPc)'),
 ('R

Now we can calculate the scores for the methods we have selected:

In [60]:
scores = {}

# Iterate over each method to calculate the LCA score
for method in my_methods:
    lca = bc.LCA(demand=my_functional_unit, method=method)
    lca.lci()
    lca.lcia()
    scores[method] = lca.score

# Convert the results to a DataFrame for easier analysis
results_df = pd.DataFrame(list(scores.items()), columns=['Method', 'Score'])

# Display the results
print(results_df)

                                               Method       Score
0   (ReCiPe 2016 v1.03, midpoint (H), acidificatio...    0.047025
1   (ReCiPe 2016 v1.03, midpoint (H), climate chan...    5.098535
2   (ReCiPe 2016 v1.03, midpoint (H), ecotoxicity:...    2.560546
3   (ReCiPe 2016 v1.03, midpoint (H), ecotoxicity:...    3.505242
4   (ReCiPe 2016 v1.03, midpoint (H), ecotoxicity:...  144.626386
5   (ReCiPe 2016 v1.03, midpoint (H), energy resou...    1.469639
6   (ReCiPe 2016 v1.03, midpoint (H), eutrophicati...    0.003982
7   (ReCiPe 2016 v1.03, midpoint (H), eutrophicati...    0.000243
8   (ReCiPe 2016 v1.03, midpoint (H), human toxici...    1.018486
9   (ReCiPe 2016 v1.03, midpoint (H), human toxici...  108.924636
10  (ReCiPe 2016 v1.03, midpoint (H), ionising rad...    0.570723
11  (ReCiPe 2016 v1.03, midpoint (H), land use, ag...    0.244537
12  (ReCiPe 2016 v1.03, midpoint (H), material res...    0.592252
13  (ReCiPe 2016 v1.03, midpoint (H), ozone deplet...    0.000002
14  (ReCiP

## 7.4 Contribution analysis
We are not only interested in the total scores but also in the contribution of each process to the total score. 

In [105]:
ca = bwa.utils.recursive_calculation_to_object((enermix.key),
    my_methods[3],
    amount=1,  # How much of the activity? (same as in our FU)
    max_level=5,  # How many levels deep in the CA do you want to go?
)
cadata=pd.DataFrame(ca)
cadata

,label,parent,score,fraction,amount,name,key
0,root,None,3.505242,1.000000,1.000000,Energy Mix Spain RICSE,"(myfg_db, energy mix SP ricse)"
1,root_c,root,3.490506,0.995796,2.000000,"battery production, lead acid, rechargeable, s...","(ei_391_cutoff, eb8e90990c77f3710bbe07cd2e866ba1)"
2,root_c_f,root_c,0.465891,0.132913,0.010000,"market for copper, cathode","(ei_391_cutoff, 8b62f30ed586a5f23611ef196cc97b93)"
3,root_c_f_b,root_c_f,0.215682,0.061531,0.001705,"copper production, cathode, solvent extraction...","(ei_391_cutoff, b3abcc9039e33e44a31c2f47a7106bb1)"
4,root_c_f_b_s,root_c_f_b,0.055992,0.015974,-0.242137,"market for sulfidic tailings, from copper mine...","(ei_391_cutoff, 8d757c61c1baccc26447217cc0a63488)"
5,root_c_f_b_s_a,root_c_f_b_s,0.055992,0.015974,-0.242137,"treatment of sulfidic tailings, from copper mi...","(ei_391_cutoff, 28a584909463426e3f43c1a8c6ce020c)"
6,root_c_f_b_y,root_c_f_b,0.043235,0.012334,-0.186510,"market for sulfidic tailings, from copper mine...","(ei_391_cutoff, a7a72ef65e0cad7c6cd45a77fd84e4d4)"
7,root_c_f_b_y_a,root_c_f_b_y,0.043235,0.012334,-0.186510,"treatment of sulfidic tailings, from copper mi...","(ei_391_cutoff, 3ccd8ad1852f5bd019a2ebf26393c029)"
8,root_c_f_l,root_c_f,0.240821,0.068703,0.006050,"electrorefining of copper, anode","(ei_391_cutoff, a594f63d4bf82f582375545495ac230a)"
9,root_c_f_l_c,root_c_f_l,0.239681,0.068378,0.006005,"market for copper, anode","(ei_391_cutoff, 6d673652ae29f74dc59fb0fc229bf6db)"


We can now plot and explore the results:

In [110]:
import plotly.graph_objects as go

# Sort the DataFrame by 'score' in descending order
cadata_sorted = cadata.sort_values(by='score', ascending=False)

# Create a list of unique labels and parents
unique_labels = list(cadata_sorted['label'].unique()) + list(cadata_sorted['parent'].unique())

# Combine and deduplicate the labels and parents
label_list = list(dict.fromkeys(unique_labels))

# Create a mapping from labels to indices
label_indices = {label: idx for idx, label in enumerate(label_list)}

# Ensure all parents are in the mapping
for parent in cadata_sorted['parent']:
    if parent not in label_indices:
        label_indices[parent] = len(label_indices)
        label_list.append(parent)

# Ensure all labels are in the mapping
for label in cadata_sorted['label']:
    if label not in label_indices:
        label_indices[label] = len(label_indices)
        label_list.append(label)

# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=[cadata_sorted[cadata_sorted['label'] == label]['name'].values[0] if label in cadata_sorted['label'].values else label for label in label_list],
    ),
    link=dict(
        source=[label_indices[parent] for parent in cadata_sorted['parent']],
        target=[label_indices[label] for label in cadata_sorted['label']],
        value=cadata_sorted['score'],
    )
)])

fig.update_layout(title_text="Sankey Diagram of LCA Scores", font_size=10, width=1000,height=1000)
fig.show()